#Object Detector using faster R-CNN

In [27]:
#relevat libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers
import numpy as np
import cv2
import pandas as pd

In [6]:
#downloading dataset
mnist=keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

#filtering dataset to keep only images with labels 0 and 1
train_filter = (train_labels == 0) | (train_labels == 1)
test_filter = (test_labels == 0) | (test_labels == 1)

train_images = train_images[train_filter]
train_labels = train_labels[train_filter]
test_images = test_images[test_filter]
test_labels = test_labels[test_filter]

In [11]:
print('Trainset with labels as 0 and 1:', len(train_images))
print('Testset with labels as 0 and 1:', len(test_images))

Trainset with labels as 0 and 1: 12665
Testset with labels as 0 and 1: 2115


In [33]:
#creating artificial bounding boxes and recording the min max coordinates
def create_bounding_boxes(images):
    bounding_boxes = []
    for image in images:
        #grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        #coordinates of non zero white pixels
        nonzero_coords = np.argwhere(image > 0)

        #calculating the bounding box coordinates
        min_y, min_x = np.min(nonzero_coords, axis=0)
        max_y, max_x = np.max(nonzero_coords, axis=0)

        #centroids
        center_x = (min_x + max_x) // 2
        center_y = (min_y + max_y) // 2

        #calculating width and height of the bounding box
        width = max_x - min_x
        height = max_y - min_y

        #limit for the bounding box
        limit = 32

        #min max coordinates of the bounding box
        top_left_x = max(0, center_x - limit // 2)
        top_left_y = max(0, center_y - limit // 2)
        bottom_right_x = min(gray_image.shape[1], center_x + limit // 2)
        bottom_right_y = min(gray_image.shape[0], center_y + limit // 2)

        bounding_boxes.append([top_left_x, top_left_y, bottom_right_x, bottom_right_y])

    return bounding_boxes

#calling te func to create boxes
train_bounding_boxes = create_bounding_boxes(train_images)
test_bounding_boxes = create_bounding_boxes(test_images)

#bounding box coordinates to arrays
train_bounding_boxes = np.array(train_bounding_boxes)
test_bounding_boxes = np.array(test_bounding_boxes)

#csv file to record the coordinates
train_df = pd.DataFrame({'Image_Index': np.arange(len(train_images)),
                         'Top_Left_X': train_bounding_boxes[:, 0],
                         'Top_Left_Y': train_bounding_boxes[:, 1],
                         'Bottom_Right_X': train_bounding_boxes[:, 2],
                         'Bottom_Right_Y': train_bounding_boxes[:, 3]})

test_df = pd.DataFrame({'Image_Index': np.arange(len(test_images)),
                        'Top_Left_X': test_bounding_boxes[:, 0],
                        'Top_Left_Y': test_bounding_boxes[:, 1],
                        'Bottom_Right_X': test_bounding_boxes[:, 2],
                        'Bottom_Right_Y': test_bounding_boxes[:, 3]})

# Save the DataFrames to CSV files
train_df.to_csv('train_bounding_boxes.csv', index=False)
test_df.to_csv('test_bounding_boxes.csv', index=False)

The above bounding boxes have same min max coordinates for all the images as they all are centered with only one feature per image.